In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
pip install mne --upgrade

In [ ]:
pip install signal-processing

In [ ]:
pip install autoreject

In [ ]:
!pip install sklearn.cross_validation

In [ ]:
pip install matplotlib==3.1.1


In [ ]:
import mne,os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from time import time
from sklearn import metrics
import pandas as pd
from glob import glob
import random
from signal_processing import *
from cmath import nan
from autoreject import get_rejection_threshold
from autoreject import AutoReject

**Start**

Data Set **A**

In [ ]:
subjectData = "/content/drive/My Drive/drdo/subjects"
cleanData = "/content/drive/My Drive/drdo/cleanData"
fname= "/content/drive/My Drive/drdo/channel_loc.csv"
TUB_montage = mne.channels.read_custom_montage(fname)
# path_file_vhdr1 = '/content/drive/My Drive/drdo/subjects/VP003/nback2.vhdr'
# path_file_vmrk1 = '/content/drive/My Drive/drdo/subjects/VP003/nback2.vmrk'
# preprocess_datasetA(path_file_vhdr1, path_file_vmrk1, "VP001", "nback1")

read_datasetA(subjectData)

In [ ]:
fname= "/content/drive/My Drive/drdo/channel_loc.csv"
# Read a montage from a file.
TUB_montage = mne.channels.read_custom_montage(fname)

def read_datasetA(subjectPath):
    
    try:
        root, dirs, files = next(os.walk(subjectPath))
        
        for folder_name in dirs:
          path_file_vhdr1 = subjectPath + '/' + folder_name + '/nback1.vhdr'
          path_file_vmrk1 = subjectPath + '/' + folder_name + '/nback1.vmrk'
          preprocess_datasetA(path_file_vhdr1, path_file_vmrk1, folder_name, "nback1")

          path_file_vhdr2 = subjectPath + '/' + folder_name + '/nback2.vhdr'
          path_file_vmrk2 = subjectPath + '/' + folder_name + '/nback2.vmrk'
          preprocess_datasetA(path_file_vhdr2, path_file_vmrk2, folder_name, "nback2")

          path_file_vhdr3 = subjectPath + '/' + folder_name + '/nback3.vhdr'
          path_file_vmrk3 = subjectPath + '/' + folder_name + '/nback3.vmrk'
          preprocess_datasetA(path_file_vhdr3, path_file_vmrk3,folder_name,"nback3")
            
    except StopIteration:
        pass
        print("Error ocurred:")
        print("Directory with dataset does not found!")
        print("Program will be terminated")
        exit(1)


In [ ]:
def preprocess_datasetA(path_file_vhdr, path_file_vmrk, subject_name , session_name):
  raw = mne.io.read_raw_brainvision(path_file_vhdr, eog=('HEOG', 'VEOG'), preload=True)  # read raw data of the subject
  raw.set_montage(TUB_montage)
  if (raw.info['nchan'] == 4):
    raw = raw.drop_channels(['EOG'])  # EOG channel is deleted
  # read annotations from a file.
  raw.info
  raw.plot(start=0, duration=6)
  mrk = mne.read_annotations(path_file_vmrk, sfreq='auto', uint16_codec=None)
  raw_downsampled = raw.copy().resample(sfreq=256)
  raw_filtered = raw_downsampled.copy().filter(l_freq=0.1 , h_freq= 45)
  raw_notch_filtered = raw_filtered.notch_filter(50, filter_length='auto', phase='zero')

  eog_projs, _ = mne.preprocessing.compute_proj_eog(raw_notch_filtered,verbose=False,n_grad=0,n_mag=0,n_eeg=1,reject=None,no_proj=True,ch_name=['HEOG', 'VEOG'])
  raw_notch_filtered.add_proj(eog_projs,remove_existing=True)
  raw_notch_filtered.apply_proj()
  raw_notch_filtered.drop_channels(['HEOG', 'VEOG']) # later remove this from filter funct
  # raw_notch_filtered.plot()  
  raw_re_referenced = mne.set_eeg_reference(raw_notch_filtered, ref_channels='average', copy=True, projection=False) 
  finData, times = raw_re_referenced[:]
  finData.plot()
  print("times")
  print(times)
  # fig, ax = plt.subplots(2)
  # # raw.plot_psd(ax=ax[0], show = False, fmax = 60)
  # finData.plot_psd(ax=ax[1], show = False, fmax=60)
  # ax[0].set_title("PSD before filtering")
  # ax[1].set_title("PSD after filtering")
  # ax[1].set_xlabel('Frequency(Hz)')
  # fig.set_tight_layout(True)
  # plt.show()
  n_components = 27 #number of components you want to fit # can be either integer which typically implies number of channels - 1 (if applied average reference)
  #     #if floating point number (0-1) fraction of total explained variance
  method = 'fastica'
  max_iter = 100
  fit_params = dict(fastica_it = 5)
  random_state = 42
  ica = mne.preprocessing.ICA(n_components = n_components,
                              method = method,
                              max_iter=max_iter,
                              random_state= random_state)
  ica.fit(finData)
  finData.load_data()
  # ica.plot_sources(finData, show_scrollbars=False)
  # ica.plot_components(sphere=1)
  ica.exclude = []
  reconst_raw = finData.copy()
  # # find which ICs match the EOG pattern
  # eog_indices, eog_scores = ica.find_bads_eog(finData, threshold = 2.5)
  # ica.exclude = eog_indices
  ica.apply(reconst_raw)
  # #print(eog_indices)
  print("after >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
  reconst_raw.plot(title="Automatic")

  events_loaded = mne.events_from_annotations(raw)

  tmin=-0.3 # when does the epoch start relative to the event onset # 300ms before the start of the event
  tmax=1.7  # when does the event end after the even onset # 500 ms from the start of the event

  # #Is a tuple containing the start of the baseline and end of the baseline
  baseline= (None, 0) #None mean begnining of the event and 0 is the start of the event 
  event_ids = {
      'Stimulus/S 16': 16,
      'Stimulus/S 48': 48,
      'Stimulus/S 64': 64,
      'Stimulus/S 80': 80,
      'Stimulus/S 96': 96,
      'Stimulus/S 112': 112,
      'Stimulus/S 128': 128,
      'Stimulus/S 144': 144,
  }

  events,event_ids = mne.events_from_annotations(reconst_raw, event_ids)

  epochs = mne.Epochs(reconst_raw, 
                      events=events,
                      event_id=event_ids,
                      tmin=tmin,tmax=tmax, 
                      baseline=baseline,
                      preload=True,event_repeated = 'drop')
  # epochs = mne.Epochs(reconst_raw,buttontarget_events,event_id=event_dict,tmin=tminmax[0],tmax=tminmax[1],
  #                           baseline=baseline,picks=picks,preload=True)
        
  # Plot the epochs' GFP plot before artefact rejection
  name = subject_name + "_" + session_name
  epochs.plot_image(title="GFP without AR ({})".format(name))

  # Use AutoReject to repair and remove epochs which are artefactual
  reject_criteria = get_rejection_threshold(epochs)
  print('Dropping epochs with rejection threshold:',reject_criteria)
  epochs.drop_bad(reject=reject_criteria)

  ar = AutoReject(thresh_method='random_search',random_state=42)
  ar.fit(epochs)
  epochs_ar, reject_log = ar.transform(epochs, return_log=True)

  print("log >>>>>>>>>>>>>>>")
  reject_log.plot('horizontal')
  print("epoches >>>>>>>>>>>>>>>>")
   # Plot the epochs' GFP after artefact rejection
  # epochs_ar.average().plot()
  epochs_ar.plot_image(title="GFP with AR ({})".format(name))
   # Display the final epochs object meta-data
  display(epochs_ar)


  export_dir = cleanData + "/" + subject_name
  try:
    os.makedirs(export_dir)
  except FileExistsError:
    pass
  try:
    filepath = export_dir + "/"+session_name +".fif"
    mne.Epochs.save(epochs,fname=filepath,overwrite=True)
  except FileExistsError:
    pass


